<a href="https://colab.research.google.com/github/harshil017/ResNet/blob/main/CatClassificationResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Loading the dataset
from google.colab import drive
drive.mount('/content/drive')
train_dir = "drive/MyDrive/CatClassificationDataSet/train"
test_dir = "drive/MyDrive/CatClassificationDataSet/test/"
val_dir = "drive/MyDrive/CatClassificationDataSet/valid/"

Mounted at /content/drive


In [2]:
#Importing the necessary Libraries
import tensorflow as tf
!pip install scikit-optimize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers, Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from skopt import gp_minimize
from skopt.space import Real
from keras.layers import Conv2D, Dense, BatchNormalization, Activation 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.4 MB/s eta 0:00:00


In [3]:
#Preprocessing the Images 
tf.random.set_seed(42)
generator = ImageDataGenerator(rescale = 1./255.,
                              horizontal_flip=True,
                              vertical_flip=False,
                              zoom_range=0.05,
                              rotation_range=25)
train_data = generator.flow_from_directory(train_dir,
                                               batch_size=32, 
                                               target_size=(224, 224), 
                                               class_mode="categorical", 
                                               seed=42)

test_data = generator.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

val_data = generator.flow_from_directory(val_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)


Found 2383 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [4]:
#Adding the Residual Block
def residual_block(x,filters,resnet_version,stride=1):
  shift = x
  #First Convolution Layer 
  conv = Conv2D(kernel_size=3,kernel_initializer='he_normal',kernel_regularizer=l2(0.001),padding='same',strides=stride,filters=3)
  #Second Convolution Layer
  conv2 =Conv2D(filters, kernel_size=3, strides=1, padding='same')
  
  if resnet_version == 2:  #Depending on the ResNet version it is convoluted first and then applying normalization and activation function or visa versa
    x =conv(x)
    x =conv2(x)
    x =BatchNormalization()(x)
    x =Activation('relu')(x)
  else:
    x =BatchNormalization()(x)
    x =Activation('relu')(x)
    x =conv(x)
    x =conv2(x)

  # Adjusting the shift for matching dimensions the residual network
  if stride != 1:
    shift = Conv2D(filters, kernel_size=1, strides=stride, padding='valid')(shift)
    shift = BatchNormalization()(shift)

  # Adding the shift i.e skip connection1
  x = layers.Add()([x, shift])
  x = layers.ReLU()(x)  

  return x   


In [8]:
#Building the Resnet architecture that is based on the ResNet building resiudual block we defined above
def resnet_model(res_version,num_blocks,input_shape,num_classes):
  inputs = tf.keras.Input(shape=input_shape)
  x = Conv2D(64,kernel_size =3,strides = 1,padding='same')(inputs)
  x = BatchNormalization()(x)
  x = layers.ReLU()(x)
  strides = 2
  print("theno of blocks",num_blocks)
  for residual_block_1 in range(num_blocks):
    x = residual_block(x,64,res_version, stride= strides if residual_block_1 == 0 else 1)
  # for residual_block_2 in range(num_blocks):
  #   x = residual_block(x,64,res_version, stride= strides if residual_block_2 == 0 else 1) 
  for residual_block_3 in range(num_blocks):
    x = residual_block(x,128,res_version, stride= strides if residual_block_3 == 0 else 1)
  for residual_block_4 in range(num_blocks):
    x = residual_block(x,256,res_version, stride= strides if residual_block_4 == 0 else 1) 
  x =  layers.AveragePooling2D(pool_size=8,padding='same')(x)
  x = layers.Flatten()(x)
  outputs = Dense(num_classes,activation='softmax')(x)
  model = Model(inputs=inputs,outputs=outputs)
  return model

In [6]:
import numpy as np
def model_call(resnetVersion,depth,input_shape,num_classes):
  print("here")
  if resnetVersion == 1:
    if (depth - 2)% 6 != 0:
      raise ValueError("Value of the depth should be of the form 6n+2")
    number_of_blocks = (depth - 2)/6
    number_of_blocks = int(number_of_blocks)
    final_model = resnet_model(resnetVersion,number_of_blocks,input_shape,num_classes)
  if resnetVersion == 2:
    if (depth - 2)% 9 != 0:
      raise ValueError("Value of the depth should be of the form 9n+2")
    number_of_blocks = (depth - 2)/9
    number_of_blocks = int(number_of_blocks) 
    final_model = resnet_model(resnetVersion,number_of_blocks,input_shape,num_classes)
  return final_model     


In [9]:
resnetVersion = 1
if resnetVersion == 1:
  depth = 32
if resnetVersion == 2:
  depth = 29  
input_shape = (224, 224, 3)
num_classes = 10
batch_size = 32
epochs = 20
model = model_call(resnetVersion,depth,input_shape,num_classes)
print
model.compile(optimizer= Adam(learning_rate = 0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
print(model)
history = model.fit (train_data,batch_size=batch_size,epochs = epochs,shuffle=True, validation_data=(val_data))
scores = model.evaluate(test_data,verbose=1)
print("Test loss",scores[0])
print("Accuracy,",scores[1])

here
theno of blocks 5
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_45 (Conv2D)             (None, 224, 224, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_25 (BatchN  (None, 224, 224, 64  256        ['conv2d_45[0][0]']              
 ormalization)                  )                                    